In [1]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from pickle import load
# from gtts.tts import gTTS
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input

In [2]:

# vgg_model = VGG16()
# vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

In [3]:
def extract_features(filename):
	vgg_model = VGG16()
	vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)
	image = load_img(filename, target_size=(224, 224))
	image = img_to_array(image)
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	image = preprocess_input(image)
	feature = vgg_model.predict(image, verbose=0)
	return feature

In [4]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [5]:
# generate caption for an image
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
      
    return in_text

In [6]:
# file = open('pkl/tokenizer.pkl', 'rb')
# tokenizer = pickle.load(file)
# file.close()

In [7]:

# model = load_model('pkl/best_model.h5', compile=False)

In [8]:
tokenizer = load(open('pkl/tokenizer.pkl', 'rb'))
model = load_model('pkl/best_model.h5', compile=False)

In [9]:
def gen(image_path):   
    max_length = 35
    features = extract_features(image_path)
    cap = predict_caption(model, features, tokenizer, max_length)
    return cap

In [10]:
# def extract_features(filename):
# 	vgg_model = VGG16()
# 	vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)
# 	image = load_img(filename, target_size=(224, 224))
# 	image = img_to_array(image)
# 	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# 	image = preprocess_input(image)
# 	feature = vgg_model.predict(image, verbose=0)
# 	return feature

In [11]:
gen('input/flickr8k/Images/1007320043_627395c3d8.jpg')

'startseq little girl climbs red climbing bridge endseq'